## Resilient Distributed Datasets (RDDs) - Lab

Resilient Distributed Datasets (RDD) are fundamental data structures of Spark. An RDD is essentially the Spark representation of a set of data, spread across multiple machines, with APIs to let you act on it. An RDD can come from any data source, e.g. text files, a database, a JSON file, etc.


## Objectives

You will be able to:

- Apply the map(func) transformation to a given function on all elements of an RDD in different partitions 
- Apply a map transformation for all elements of an RDD 
- Compare the difference between a transformation and an action within RDDs 
- Use collect(), count(), and take() actions to trigger spark transformations  
- Use filter to select data that meets certain specifications within an RDD 
- Set number of partitions for parallelizing RDDs 
- Create RDDs from Python collections 


## What are RDDs? 

To get a better understanding of RDDs, let's break down each one of the components of the acronym RDD:

Resilient: RDDs are considered "resilient" because they have built-in fault tolerance. This means that even if one of the nodes goes offline, RDDs will be able to restore the data. This is already a huge advantage compared to standard storage. If a standard computer dies while performing an operation, all of its memory will be lost in the process. With RDDs, multiple nodes can go offline, and the action will still be held in working memory.

Distributed: The data is contained on multiple nodes of a cluster-computing operation. It is efficiently partitioned to allow for parallelism.

Dataset: The dataset has been * partitioned * across the multiple nodes. 

RDDs are the building block upon which more high-level Spark operations are based upon. Chances are, if you are performing an action using Spark, the operation involves RDDs. 



Key Characteristics of RDDs:

- Immutable: Once an RDD is created, it cannot be modified. 
- Lazily Evaluated: RDDs will not be evaluated until an action is triggered. Essentially, when RDDs are created, they are programmed to perform some action, but that function will not get activated until it is explicitly called. The reason for lazy evaluation is that allows users to organize the actions of their Spark program into smaller actions. It also saves unnecessary computation and memory load.
- In-Memory: The operations in Spark are performed in-memory rather than in the database. This is what allows Spark to perform fast operations with very large quantities of data.




### RDD Transformations vs Actions

In Spark, we first create a __base RDD__ and then apply one or more transformations to that base RDD following our processing needs. Being immutable means, **once an RDD is created, it cannot be changed**. As a result, **each transformation of an RDD creates a new RDD**. Finally, we can apply one or more **actions** to the RDDs. Spark uses lazy evaluation, so transformations are not actually executed until an action occurs.


<img src="./images/rdd_diagram.png" width=500>

### Transformations

Transformations create a new dataset from an existing one by passing each dataset element through a function and returning a new RDD representing the results. In short, creating an RDD from an existing RDD is ‘transformation’.
All transformations in Spark are lazy. They do not compute their results right away. Instead, they just remember the transformations applied to some base dataset (e.g. a file). The transformations are only computed when an action requires a result that needs to be returned to the driver program.
A transformation is an RDD that returns another RDD, like map, flatMap, filter, reduceByKey, join, cogroup, etc.

### Actions
Actions return final results of RDD computations. Actions trigger execution using lineage graph to load the data into original RDD and carry out all intermediate transformations and return the final results to the driver program or writes it out to the file system. An action returns a value (to a Spark driver - the user program).

Here are some key transformations and actions that we will explore.


| Transformations   | Actions       |
|-------------------|---------------|
| map(func)         | reduce(func)  |
| filter(func)      | collect()     |
| groupByKey()      | count()       |
| reduceByKey(func) | first()       |
| mapValues(func)   | take()        |
| sample()          | countByKey()  |
| distinct()        | foreach(func) |
| sortByKey()       |               |


Let's see how transformations and actions work through a simple example. In this example, we will perform several actions and transformations on RDDs in order to obtain a better understanding of Spark processing. 

### Create a Python collection 

We need some data to start experimenting with RDDs. Let's create some sample data and see how RDDs handle it. To practice working with RDDs, we're going to use a simple Python list.

- Create a Python list `data` of integers between 1 and 1000 using the `range()` function. 
- Sanity check: confirm the length of the list (it should be 1000)

In [1]:
data = list(range(1,1001))
len(data)

# 1000

1000

### Initialize an RDD

When using Spark to make computations, datasets are treated as lists of entries. Those lists are split into different partitions across different cores or different computers. Each list of data held in memory is a partition of the RDD. The reason why Spark is able to make computations far faster than other big data processing languages is that it allows all data to be stored __in-memory__, which allows for easy access to the data and, in turn, high-speed processing. Here is an example of how the alphabet might be split into different RDDs and held across a distributed collection of nodes:

<img src ="./images/partitions_1.png" width ="500">  
To initialize an RDD, first import `pyspark` and then create a SparkContext assigned to the variable `sc`. Use `'local[*]'` as the master.

In [2]:
import pyspark
sc = pyspark.SparkContext('local[*]')

Once you've created the SparkContext, you can use the `.parallelize()` method to create an RDD that will distribute the list of numbers across multiple cores. Here, create one called `rdd` with 10 partitions using `data` as the collection you are parallelizing.

In [3]:
rdd = sc.parallelize(data, numSlices=10)
print(type(rdd))
# <class 'pyspark.rdd.RDD'>

<class 'pyspark.rdd.RDD'>


Determine how many partitions are being used with this RDD with the `.getNumPartitions()` method.

In [4]:
rdd.getNumPartitions()
# 10

10

### Basic descriptive RDD actions

Let's perform some basic operations on our RDD. In the cell below, use the methods:
* `count`: returns the total count of items in the RDD 
* `first`: returns the first item in the RDD
* `take`: returns the first `n` items in the RDD
* `top`: returns the top `n` items
* `collect`: returns everything from your RDD


It's important to note that in a big data context, calling the collect method will often take a very long time to execute and should be handled with care!

In [5]:
# count
rdd.count()

1000

In [6]:
# first
rdd.first()

1

In [7]:
# take
rdd.take(3)

[1, 2, 3]

In [8]:
# top
rdd.top(10)

[1000, 999, 998, 997, 996, 995, 994, 993, 992, 991]

In [9]:
# collect
rdd.collect()

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185

## Map functions

Now that you've been working a little bit with RDDs, let's make this a little more interesting. Imagine you're running a hot new e-commerce startup called BuyStuff, and you're trying to track of how much it charges customers from each item sold. In the next cell, we're going to create simulated data by multiplying the values 1-1000 with a random number from 0-1.

In [10]:
import random
import numpy as np

nums = np.array(range(1, 1001))
sales_figures = nums * np.random.rand(1000)
sales_figures

array([8.99738734e-01, 1.23058366e+00, 1.54902972e+00, 3.80758152e+00,
       2.34904253e+00, 9.22410543e-01, 5.76873232e-01, 6.09336492e+00,
       3.35722311e+00, 3.18693384e+00, 9.81550513e+00, 9.74909010e+00,
       3.59517771e+00, 4.16347541e+00, 2.14668817e+00, 7.57909156e+00,
       6.43507829e+00, 9.48363798e+00, 8.43814336e+00, 1.98254320e+01,
       1.14750499e+01, 3.59400316e+00, 1.55016799e+01, 7.27700419e+00,
       7.82705721e+00, 2.01709456e+01, 1.51405921e+01, 8.45342014e-01,
       1.56055888e+01, 2.61625371e+01, 2.39735187e+01, 2.58496331e+00,
       7.82294496e+00, 7.79171211e+00, 2.94357030e+01, 1.13042974e+00,
       1.09065557e+00, 4.85309023e+00, 1.40457709e+01, 2.10437696e+01,
       1.04453514e+01, 3.08199251e+01, 4.12952007e+01, 3.11489547e+00,
       1.04992553e+01, 4.48726013e+01, 1.53697759e+01, 4.62824445e+01,
       4.59786159e+01, 2.88750292e+01, 4.59177169e+01, 2.38080616e+01,
       1.28099206e+01, 8.46673984e+00, 3.53295950e+01, 2.86564667e+01,
      

We now have sales prices for 1000 items currently for sale at BuyStuff. Now create an RDD called `price_items` using the newly created data with 10 slices. After you create it, use one of the basic actions to see what's in the RDD.

In [11]:
price_items = sc.parallelize(sales_figures, numSlices=10)
price_items.count()


1000

Now let's perform some operations on this simple dataset. To begin with, create a function that will take into account how much money BuyStuff will receive after sales tax has been applied (assume a sales tax of 8%). To make this happen, create a function called `sales_tax()` that returns the amount of money our company will receive after the sales tax has been applied. The function will have this parameter:

* `item`: (float) number to be multiplied by the sales tax.


Apply that function to the rdd by using the `.map()` method and assign it to a variable `revenue_minus_tax`

In [24]:
def sales_tax(num):
    return num * 0.92

revenue_minus_tax = price_items.map(sales_tax)

Remember, Spark has __lazy evaluation__, which means that the `sales_tax()` function is a transformer that is not executed until you call an action. Use one of the collection methods to execute the transformer now a part of the RDD and observe the contents of the `revenue_minus_tax` rdd.

In [25]:
revenue_minus_tax.top(10)


[899.7793686291817,
 865.1182781915198,
 838.2021374105992,
 832.2414065768047,
 830.9623312417458,
 816.1095311824387,
 807.8449645685038,
 800.4630780956381,
 798.2555352900424,
 788.0801074993623]

### Lambda Functions

Note that you can also use lambda functions if you want to quickly perform simple operations on data without creating a function. Let's assume that BuyStuff has also decided to offer a 10% discount on all of their items on the pre-tax amounts of each item. Use a lambda function within a `.map()` method to apply the additional 10% loss in revenue for BuyStuff and assign the transformed RDD to a new RDD called `discounted`.

In [20]:
discounted = price_items.map(lambda x: x* 0.9)

In [21]:
discounted.take(10)

[0.8097648605398251,
 1.1075252901037431,
 1.3941267501929895,
 3.42682336514889,
 2.114138274005914,
 0.8301694883733693,
 0.5191859090995876,
 5.4840284260151435,
 3.0215007990749867,
 2.8682404548717497]

## Chaining Methods

You are also able to chain methods together with Spark. In one line, remove the tax and discount from the revenue of BuyStuff and use a collection method to see the 15 costliest items.

In [28]:
price_items.map(sales_tax).top(15)

[899.7793686291817,
 865.1182781915198,
 838.2021374105992,
 832.2414065768047,
 830.9623312417458,
 816.1095311824387,
 807.8449645685038,
 800.4630780956381,
 798.2555352900424,
 788.0801074993623,
 772.220953307471,
 756.9336816644519,
 749.3963479960872,
 747.5168275987348,
 742.8345162138526]

## RDD Lineage


We are able to see the full lineage of all the operations that have been performed on an RDD by using the `RDD.toDebugString()` method. As your transformations become more complex, you are encouraged to call this method to get a better understanding of the dependencies between RDDs. Try calling it on the `discounted` RDD to see what RDDs it is dependent on.

In [29]:
discounted.toDebugString()

b'(10) PythonRDD[16] at RDD at PythonRDD.scala:53 []\n |   ParallelCollectionRDD[5] at parallelize at PythonRDD.scala:195 []'

### Map vs. Flatmap

Depending on how you want your data to be outputted, you might want to use `.flatMap()` rather than a simple `.map()`. Let's take a look at how it performs operations versus the standard map. Let's say we wanted to maintain the original amount BuyStuff receives for each item as well as the new amount after the tax and discount are applied. Create a map function that will return a tuple with (original price, post-discount price).

In [30]:
mapped =price_items.map(lambda x :(x, sales_tax(x)*.9))
print(mapped.count())
print(mapped.take(10))

1000
[(0.899738733933139, 0.744983671696639), (1.2305836556708256, 1.0189232668954438), (1.549029722436655, 1.2825966101775503), (3.8075815168320997, 3.1526774959369788), (2.3490425266732378, 1.9450072120854411), (0.922410542637077, 0.7637559293034998), (0.5768732323328751, 0.4776510363716206), (6.093364917794604, 5.045306151933933), (3.3572231100833183, 2.7797807351489876), (3.1869338387463886, 2.63878121848201)]


Note that we have 1000 tuples created to our specification. Let's take a look at how `.flatMap()` differs in its implementation. Use the `.flatMap()` method with the same function you created above.

In [31]:
flat_mapped = price_items.flatMap(lambda x :(x, sales_tax(x)*.9))
print(flat_mapped.count())
print(flat_mapped.take(10))

2000
[0.899738733933139, 0.744983671696639, 1.2305836556708256, 1.0189232668954438, 1.549029722436655, 1.2825966101775503, 3.8075815168320997, 3.1526774959369788, 2.3490425266732378, 1.9450072120854411]


Rather than being represented by tuples, all of the  values are now on the same level. When we are trying to combine different items together, it is sometimes necessary to use `.flatMap()` rather than `.map()` in order to properly reduce to our specifications. This is not one of those instances, but in the upcoming lab, you just might have to use it.

## Filter
After meeting with some external consultants, BuyStuff has determined that its business will be more profitable if it focuses on higher ticket items. Now, use the `.filter()` method to select items that bring in more than $300 after tax and discount have been removed. A filter method is a specialized form of a map function that only returns the items that match a certain criterion. In the cell below:
* use a lambda function within a `.filter()` method to meet the consultant's suggestion's specifications. set `RDD = selected_items`
* calculate the total number of items remaining in BuyStuff's inventory

In [32]:
# use the filter function
selected_items = price_items.filter(lambda x: sales_tax(x)*.90>300)

# calculate total remaining in inventory 
selected_items.count()

270

## Reduce

Reduce functions are where you are in some way combing all of the variables that you have mapped out. Here is an example of how a reduce function works when the task is to sum all values:

<img src = "./images/reduce_function.png" width = "600">  


As you can see, the operation is performed within each partition first, after which, the results of the computations in each partition are combined to come up with one final answer.  

Now it's time to figure out how much money BuyStuff would make from selling one of all of its items after they've reduced their inventory. Use the `.reduce()` method with a lambda function to add up all of the values in the RDD. Your lambda function should have two variables. 

In [33]:
selected_items.reduce(lambda x, y: x+ y)

150745.12468034515

The time has come for BuyStuff to open up shop and start selling its goods. It only has one of each item, but it's allowing 50 lucky users to buy as many items as they want while they remain in stock. Within seconds, BuyStuff is sold out. Below, you'll find the sales data in an RDD with tuples of (user, item bought).

In [34]:
import random
random.seed(42)
# generating simulated users that have bought each item
sales_data = selected_items.map(lambda x: (random.randint(1, 50), x))

sales_data.take(7)

[(25, 369.2639950128622),
 (5, 375.63369072837236),
 (7, 363.25674899916373),
 (48, 370.04525873873865),
 (25, 392.8843974783909),
 (9, 384.3335883717267),
 (34, 438.3390550360844)]

It's time to determine some basic statistics about BuyStuff users.

Let's start off by creating an RDD that determines how much each user spent in total.
To do this we can use a method called `.reduceByKey()` to perform reducing operations while grouping by keys. After you have calculated the total, use the `.sortBy()` method on the RDD to rank the users from the highest spending to the least spending. 

In [37]:
# calculate how much each user spent
total_spent = sales_data.reduceByKey(lambda x, y: x + y)
total_spent.take(10)

[(50, 2934.231577812499),
 (20, 3514.5554698060832),
 (30, 477.06204113742257),
 (10, 1949.6530710880745),
 (40, 2499.9182491880865),
 (31, 4347.064807731983),
 (1, 4679.174891319223),
 (11, 3803.509573181673),
 (41, 2780.507487908285),
 (21, 1161.662882261354)]

In [41]:
# sort the users from highest to lowest spenders
total_spent.sortBy(lambda x: x[1],ascending = False).collect()

[(5, 7228.255084173294),
 (17, 5769.601316181468),
 (18, 5125.423353055746),
 (39, 4931.280390372646),
 (9, 4766.1262396921),
 (1, 4679.174891319223),
 (19, 4678.396754104098),
 (25, 4400.725531204281),
 (31, 4347.064807731983),
 (29, 4163.364540639334),
 (45, 3958.346392527314),
 (49, 3883.0942731418877),
 (33, 3852.1929828474504),
 (42, 3825.274159207469),
 (11, 3803.509573181673),
 (26, 3689.354018176904),
 (7, 3671.746270411086),
 (2, 3536.8334633024856),
 (20, 3514.5554698060832),
 (3, 3390.6278898679275),
 (12, 3316.7225629713157),
 (36, 3198.9051615550898),
 (27, 3024.3531221292524),
 (50, 2934.231577812499),
 (37, 2890.487730250892),
 (43, 2873.966620090375),
 (4, 2826.6285679999046),
 (41, 2780.507487908285),
 (44, 2682.722026434513),
 (15, 2668.1322328042356),
 (6, 2629.6993546490075),
 (40, 2499.9182491880865),
 (23, 2491.9210097145065),
 (47, 2479.519339578107),
 (13, 2365.2295136942453),
 (14, 2220.799071865717),
 (22, 2180.146300561363),
 (28, 2164.689436521778),
 (10, 19

Next, let's determine how many items were bought per user. This can be solved in one line using an RDD method. After you've counted the total number of items bought per person, sort the users from most number of items bought to least number of items. Time to start a customer loyalty program!

In [43]:
total_items = sales_data.countByKey()
sorted(total_items.items(),key=lambda kv:kv[1],reverse=True)

defaultdict(int,
            {12: 6,
             10: 5,
             45: 6,
             30: 8,
             28: 7,
             1: 6,
             4: 7,
             39: 8,
             44: 5,
             32: 9,
             19: 8,
             27: 7,
             35: 6,
             21: 5,
             26: 10,
             2: 5,
             38: 4,
             31: 8,
             13: 13,
             14: 3,
             41: 3,
             7: 6,
             9: 5,
             33: 6,
             50: 9,
             47: 6,
             34: 4,
             18: 9,
             16: 5,
             43: 4,
             15: 5,
             48: 1,
             22: 5,
             42: 3,
             11: 3,
             24: 5,
             36: 5,
             40: 6,
             37: 5,
             49: 6,
             25: 4,
             20: 3,
             29: 4,
             46: 2,
             17: 2,
             8: 4,
             3: 4,
             5: 4,
             6: 4,
          

### Additional Reading

- [The original paper on RDDs](https://cs.stanford.edu/~matei/papers/2012/nsdi_spark.pdf)
- [RDDs in Apache Spark](https://data-flair.training/blogs/create-rdds-in-apache-spark/)
- [Programming with RDDs](https://runawayhorse001.github.io/LearningApacheSpark/rdd.html)
- [RDD Transformations and Actions Summary](https://www.analyticsvidhya.com/blog/2016/10/using-pyspark-to-perform-transformations-and-actions-on-rdd/)

## Summary

In this lab we went through a brief introduction to RDD creation from a Python collection, setting a number of logical partitions for an RDD and extracting lineage. We also used transformations and actions to perform calculations across RDDs on a distributed setup. In the next lab, you'll get the chance to apply these transformations on different books to calculate word counts and various statistics.
